In [12]:
import torch
from Nets import Net, NetNoise
from functions import train, test
from dataloaders import MNIST, MNIST_train_random_lebels, dead_leaves_squares_pretraining
import numpy as np
import torch.optim as optim
import matplotlib.pyplot as plt
import cv2

torch.cuda.is_available()

True

In [62]:
from torch.utils.data import Dataset
import os
import json
from skimage import io
import torchvision
from PIL import Image

resize_image = True

transform_array = []
if resize_image:
    transform_array.append(
        torchvision.transforms.Resize((96,96)) # PIL Image or Tensor
    )

transform_array += [
    torchvision.transforms.ToTensor(),
    torchvision.transforms.RandomResizedCrop(64, scale=(0.08, 1)), # tensor pil
    torchvision.transforms.RandomHorizontalFlip(), # PIL Image or Tensor
    torchvision.transforms.ColorJitter(0.4, 0.4, 0.4, 0.4), # PIL Image or Tensor
    torchvision.transforms.RandomGrayscale(p=0.2),
    torchvision.transforms.Normalize(
        (0.44087801806139126, 0.42790631331699347, 0.3867879370752931),
        (0.26826768628079806, 0.2610450402318512, 0.26866836876860795),
    ),
]

transform = torchvision.transforms.Compose(transform_array)

class ImageNet100(Dataset):
    def __init__(self, data_root, transform=None):
        train_folders = ["train.X1","train.X2","train.X3","train.X4"]
        f = open("labels.json")
        labels_dict = json.load(f)
        self.data_root = data_root
        self.images = []
        self.labels = []
        self.transform = transform

        for train_folder in train_folders:
            for label in os.listdir(data_root+'\\'+train_folder):
                for img in os.listdir(data_root+'\\'+train_folder+'\\'+label):
                    self.images.append(data_root+'\\'+train_folder+'\\'+label+'\\'+img)
                    self.labels.append(labels_dict[label])
        self.labels = torch.tensor(self.labels)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
            # x = io.imread("E:\\Datasets\\Imagenet100\\train.X1\\n01440764\\n01440764_10026.JPEG").T
            # x = torchvision.io.read_image("E:\\Datasets\\Imagenet100\\train.X1\\n01440764\\n01440764_10026.JPEG")
            x = Image.open("E:\\Datasets\\Imagenet100\\train.X1\\n01440764\\n01440764_10026.JPEG")
            # x = cv2.imread("E:\\Datasets\\Imagenet100\\train.X1\\n01440764\\n01440764_10026.JPEG")
            # print(x.shape)

            if self.transform:
                x = self.transform(x)

            return x, self.labels[idx]

In [14]:
# io.imread("E:\\Datasets\\Imagenet100\\train.X1\\n01440764\\n01440764_10026.JPEG").T

In [15]:
# torchvision.io.read_image("E:\\Datasets\\Imagenet100\\train.X1\\n01440764\\n01440764_10026.JPEG")

In [16]:
# Image.open("E:\\Datasets\\Imagenet100\\train.X1\\n01440764\\n01440764_10026.JPEG")

In [17]:
# import cv2
# cv2.imread("E:\\Datasets\\Imagenet100\\train.X1\\n01440764\\n01440764_10026.JPEG")

In [18]:
# plt.imread("E:\\Datasets\\Imagenet100\\train.X1\\n01440764\\n01440764_10026.JPEG").T

In [64]:
print('-'*50, 'pretraining', '-'*50)
train_loader_random_lebels = dead_leaves_squares_pretraining()

model = NetNoise()
optimizer = optim.SGD(model.parameters(), lr=0.1)
# train_losses = train(model, optimizer, train_loader_random_lebels)
# plt.plot(train_losses)

model.load_state_dict(torch.load("pretrained_dead_leaves_squares.pt"))

-------------------------------------------------- pretraining --------------------------------------------------
Loading data from data/dead_leaves-squares/ as imagefolder


<All keys matched successfully>

In [71]:
next(iter(train_loader_random_lebels))[0][0].shape

torch.Size([3, 64, 64])

In [65]:
print('-'*50, 'training only last layer', '-'*50)
dl = ImageNet100("E:\Datasets\Imagenet100", transform)
train_loader = torch.utils.data.DataLoader(dl, batch_size=1, num_workers=2, shuffle=False)

newModel = model.return_pretrained_model(100)
# train_losses = train(model, optimizer, train_loader)
# test(model, test_loader)

-------------------------------------------------- training only last layer --------------------------------------------------


In [73]:
model

NetNoise(
  (conv1): Conv2d(3, 10, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2_drop): Dropout2d(p=0.5, inplace=False)
  (fc1): Linear(in_features=3380, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=105, bias=True)
)

In [72]:
newModel

Sequential(
  (0): Conv2d(3, 10, kernel_size=(5, 5), stride=(1, 1))
  (1): Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1))
  (2): Dropout2d(p=0.5, inplace=False)
  (3): Linear(in_features=3380, out_features=50, bias=True)
  (4): Linear(in_features=50, out_features=100, bias=True)
)

In [66]:
print(dl[0][0].shape)

newModel(dl[0][0])


torch.Size([3, 64, 64])


RuntimeError: mat1 and mat2 shapes cannot be multiplied (1120x56 and 3380x50)

In [ ]:
dl[0][0]

TypeError: Unexpected type <class 'numpy.ndarray'>

In [ ]:
type(dl[0][0])

torch.Tensor

In [ ]:
train_loader_random_lebels.dataset[0][0].shape

torch.Size([3, 64, 64])

In [ ]:
model

NetNoise(
  (conv1): Conv2d(3, 10, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2_drop): Dropout2d(p=0.5, inplace=False)
  (fc1): Linear(in_features=3380, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=105, bias=True)
)

In [ ]:
import torchvision
train_path = "E:\Datasets\Imagenet100\\train.X1"

dataset = torchvision.datasets.ImageFolder(
    train_path,
    transform=transform)

train_loader = torch.utils.data.DataLoader(dataset, batch_size=1, num_workers=2, shuffle=False)

In [ ]:
dataset[0][0].shape

torch.Size([3, 64, 64])

In [ ]:
model(dataset[0][0])

c:\Users\QbaSo\AppData\Local\Programs\Python\Python39\lib\site-packages\torch\nn\functional.py:1338: UserWarning: dropout2d: Received a 3D input to dropout2d and assuming that channel-wise 1D dropout behavior is desired - input is interpreted as shape (N, C, L), where C is the channel dim. This behavior will change in a future release to interpret the input as one without a batch dimension, i.e. shape (C, H, W). To maintain the 1D channel-wise dropout behavior, please switch to using dropout1d instead.
  warnings.warn("dropout2d: Received a 3D input to dropout2d and assuming that channel-wise "


RuntimeError: mat1 and mat2 shapes cannot be multiplied (20x169 and 3380x50)

In [ ]:
dataset

Dataset ImageFolder
    Number of datapoints: 32500
    Root location: E:\Datasets\Imagenet100\train.X1

In [ ]:
dl = ImageNet100("E:\Datasets\Imagenet100")
train_loader = torch.utils.data.DataLoader(dataset, batch_size=1, num_workers=2, shuffle=False)

In [ ]:
# unique_labels = np.unique(dl.labels)

# labels_dict = dict()

# for i in range(len(unique_labels)):
#     labels_dict[unique_labels[i]] = i
    
# f = open("labels.json", "w")

# json.dump(labels_dict, f)
# f.close()


In [ ]:
train_loader = torch.utils.data.DataLoader(dl, batch_size=1, num_workers=2, shuffle=False)

In [ ]:
train_loader.dataset[0]

(array([[[255, 255, 255],
         [255, 255, 255],
         [255, 255, 255],
         ...,
         [255, 255, 255],
         [255, 255, 255],
         [255, 255, 255]],
 
        [[255, 255, 255],
         [255, 255, 255],
         [255, 255, 255],
         ...,
         [255, 255, 255],
         [255, 255, 255],
         [255, 255, 255]],
 
        [[255, 255, 255],
         [255, 255, 255],
         [255, 255, 255],
         ...,
         [255, 255, 255],
         [255, 255, 255],
         [255, 255, 255]],
 
        ...,
 
        [[255, 255, 255],
         [255, 255, 255],
         [255, 255, 255],
         ...,
         [255, 255, 255],
         [255, 255, 255],
         [255, 255, 255]],
 
        [[255, 255, 255],
         [255, 255, 255],
         [255, 255, 255],
         ...,
         [255, 255, 255],
         [255, 255, 255],
         [255, 255, 255]],
 
        [[255, 255, 255],
         [255, 255, 255],
         [255, 255, 255],
         ...,
         [255, 255, 255],
  

In [ ]:
iter(train_loader)

In [ ]:
train_loader_random_lebels = dead_leaves_squares_pretraining()

Loading data from data/dead_leaves-squares/ as imagefolder


In [ ]:
train_loader_random_lebels.dataset[0][0].shape

torch.Size([3, 64, 64])

In [ ]:
model = NetNoise()

In [ ]:
newModel = model.return_pretrained_model(100)

In [ ]:
model

NetNoise(
  (conv1): Conv2d(3, 10, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2_drop): Dropout2d(p=0.5, inplace=False)
  (fc1): Linear(in_features=3380, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=105, bias=True)
)

In [ ]:
import torch.nn as nn
newModel.append(nn.Linear())

Sequential(
  (0): Conv2d(3, 10, kernel_size=(5, 5), stride=(1, 1))
  (1): Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1))
  (2): Dropout2d(p=0.5, inplace=False)
  (3): Linear(in_features=3380, out_features=50, bias=True)
)

In [ ]:
import torch.nn as nn
last_layer_in_features = list(list(model.children())[-1].parameters())[0].shape[1]
no_classes = 100

newModel = nn.Sequential(
    *list(model.children())[:-1],
    nn.Linear(last_layer_in_features, no_classes))

In [ ]:
newModel

Sequential(
  (0): Conv2d(3, 10, kernel_size=(5, 5), stride=(1, 1))
  (1): Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1))
  (2): Dropout2d(p=0.5, inplace=False)
  (3): Linear(in_features=3380, out_features=50, bias=True)
  (4): Linear(in_features=50, out_features=100, bias=True)
)

In [ ]:
model

NetNoise(
  (conv1): Conv2d(3, 10, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2_drop): Dropout2d(p=0.5, inplace=False)
  (fc1): Linear(in_features=3380, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=105, bias=True)
)

In [ ]:
list(list(model.children())[-1].parameters())[0].shape[1]

50